## GPT를 활용하여 식물의 상태 분석하기
- 식물의 기본 정보와 분석 데이터를 Chat GPT에 전송하여, 미리 정의된 출력 형식에 맞춘 응답을 받는다.
  - 인풋 데이터 : 1차 분석결과, 식물 환경 센서 데이터, 사용자가 입력한 식물 환경 정보 등
    - 1차 분석결과: 식물의 사진을 식물 질병 예측 모델(CNN)에 입력하여 식물의 질병을 예측한다.
    - 식물 환경 센서 데이터 : 라즈베리파이로 측정한 센서 데이터 값
    - 사용자가 입력한 식물 환경 정보 : 사용자가 직접 입력한 식물의 상태

  - output 형식 : json 형태
    - 구현 방식
      - 방식 1 : Text generation의 JSON mode 사용
      - 방식 2 : Function calling 사용

    

### 방식 1 : Text generation의 JSON mode

#### 처리 순서
1. 인풋 데이터 텍스트화
2. Chat GPT API 요청
  - 사전 준비 과정
    - 1. 어시스턴트 생성
    - 2. JSON mode 설정
    - 3. 출력 포멧 설정
    - 4. 어시스턴트 기본 쓰레드 설정
3. 응답 파싱 및 가공
\* 인풋데이터는 원시데이터의 형태가 아닌 1차 가공된(날짜별 평균 및 최저 데이터) 데이터로 사전 작업해줘야한다.

##### 1. 인풋 데이터 텍스트화
- 인풋 데이터를 텍스트화 한다.
  - 인풋 데이터 : 1차 분석결과, 식물 환경 센서 데이터, 사용자가 입력한 식물 환경 정보 등
  - 해당 노트북에선 더미 데이터를 만들어 사용한다.

In [2]:
# 더미데이터

# 1차 분석 결과 데이터
predicted_disease = "healthy"

# 식물 환경 센서 데이터
env_data = {
  "2024-06-19": {
    "highest temperature": 41,
    "lowest temperature": 31,
    "average temperature": 38,
    "average humidity": "62%",
    "average illuminance": "7500 lux"
  },
  "2024-06-20": {
    "highest temperature": 42,
    "lowest temperature": 32,
    "average temperature": 39,
    "average humidity": "63%",
    "average illuminance": "7600 lux"
  },
  "2024-06-21": {
    "highest temperature": 43,
    "lowest temperature": 33,
    "average temperature": 40,
    "average humidity": "64%",
    "average illuminance": "7700 lux"
  },
  "2024-06-22": {
    "highest temperature": 44,
    "lowest temperature": 34,
    "average temperature": 41,
    "average humidity": "65%",
    "average illuminance": "7800 lux"
  },
  "2024-06-23": {
    "highest temperature": 45,
    "lowest temperature": 35,
    "average temperature": 42,
    "average humidity": "66%",
    "average illuminance": "7900 lux"
  },
  "2024-06-24": {
    "highest temperature": 46,
    "lowest temperature": 36,
    "average temperature": 43,
    "average humidity": "67%",
    "average illuminance": "8000 lux"
  },
  "2024-06-25": {
    "highest temperature": 47,
    "lowest temperature": 37,
    "average temperature": 44,
    "average humidity": "68%",
    "average illuminance": "8100 lux"
  },
  "2024-06-26": {
    "highest temperature": 48,
    "lowest temperature": 38,
    "average temperature": 45,
    "average humidity": "69%",
    "average illuminance": "8200 lux"
  },
  "2024-06-27": {
    "highest temperature": 49,
    "lowest temperature": 39,
    "average temperature": 46,
    "average humidity": "70%",
    "average illuminance": "8300 lux"
  },
  "2024-06-28": {
    "highest temperature": 50,
    "lowest temperature": 40,
    "average temperature": 47,
    "average humidity": "71%",
    "average illuminance": "8400 lux"
  }
}

# 사용자가 입력한 식물 환경 정보
user_input = {
  "Watering per day": "500ml",
  "wind hours per day": 2,
  "lighting hours per day": 3
}

In [6]:
def data_to_text(predicted_disease, env_data, user_input):
  # "다음 정보를 활용하여 식물을 분석해줘"
  result = "Analyze your plant using the following information.\n"

  # 1차 분석 결과 데이터
  result += f"\n1. Predicted disease: {predicted_disease}\n"

  # 식물 환경 센서 데이터
  result += f"\n2. Environment sensor data: \n"
  for date, sensor_data in env_data.items():
    result += f"Date: {date}\n"
    for key, value in sensor_data.items():
      result += f"- {key}: {value}\n"

  # 사용자가 입력한 식물 환경 정보
  result += f"\n3. User input: \n"
  for key, value in user_input.items():
    result += f"- {key}: {value}\n"

  return result

In [7]:
data_to_text(predicted_disease, env_data, user_input)

'Analyze your plant using the following information.\n\n1. Predicted disease: healthy\n\n2. Environment sensor data: \nDate: 2024-06-19\n- highest temperature: 41\n- lowest temperature: 31\n- average temperature: 38\n- average humidity: 62%\n- average illuminance: 7500 lux\nDate: 2024-06-20\n- highest temperature: 42\n- lowest temperature: 32\n- average temperature: 39\n- average humidity: 63%\n- average illuminance: 7600 lux\nDate: 2024-06-21\n- highest temperature: 43\n- lowest temperature: 33\n- average temperature: 40\n- average humidity: 64%\n- average illuminance: 7700 lux\nDate: 2024-06-22\n- highest temperature: 44\n- lowest temperature: 34\n- average temperature: 41\n- average humidity: 65%\n- average illuminance: 7800 lux\nDate: 2024-06-23\n- highest temperature: 45\n- lowest temperature: 35\n- average temperature: 42\n- average humidity: 66%\n- average illuminance: 7900 lux\nDate: 2024-06-24\n- highest temperature: 46\n- lowest temperature: 36\n- average temperature: 43\n- a